In [1]:
from datetime import datetime, timedelta
from iexfinance.stocks import get_historical_data
from iexfinance.refdata import get_symbols
import os
import pandas as pd

In [ ]:
token = os.getenv("IEX_TOKEN")

ticker = "ILMN"


end_date = datetime.now()
start_date = end_date + timedelta(-365)


df = get_historical_data(ticker, start_date, end_date,  output_format='pandas', token=token)
df = df['close'].to_frame()
df.columns = ['ILMN']


In [ ]:
tickers = ['VFIAX', 'VTSAX', 'VEXAX', 'VIGAX', 'VVIAX', 'VIMAX', 'VMGMX', 'VMVAX', 'VSMAX', 'VSGAX', 'VSIAX', 'VTMGX', 'VFWAX', 'VTIAX', 'VEMAX', 'VGSLX', 'VBTLX', 'VTEAX', 'VSBSX', 'VSIGX', 'VLGSX', 'VBIRX', 'VFTAX', 'VTABX']
end_date = datetime.now()
start_date = end_date + timedelta(-365)
stocks_df = df

for ticker in tickers:
    print(ticker)
    try:
        stock_df = get_historical_data(ticker, start_date, end_date,  output_format='pandas', token=token)
        stock_df = stock_df['close'].to_frame()
        stock_df.columns = tick
        stocks_df = pd.concat([stocks_df, stock_df], axis=1, join='inner')
    except Exception as e:
        print(e)
        